In [1]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch

import numpy as np
import cv2

import os  
from pathlib import Path
import sys

import matplotlib.pyplot as plt

from einops import rearrange

import glob
from sklearn.model_selection import train_test_split

In [2]:
path = Path(os.getcwd())
parent = path.parent
parent = str(parent)
path = str(path)
print(path)
sys.path.insert(1, path)
sys.path.insert(1, parent)

/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio


In [3]:
from dataloader_dev.dataset_generator_class import ImageDatasetGenerator
from models.CvT_model import CvT

In [4]:
# Transforms -- We can add augmentations here
transform_images = transforms.Compose([ 
                        transforms.ToTensor()       
                        #transforms.Normalize(mean = mean, std = std)
                                    ])

transform_guidance = transforms.Compose([ 
                        transforms.ToTensor()
                                    ])

In [5]:
# Folder paths
image_folder = "/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio/training_data/training_images_cropped"
isd_map_folder = "/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio/training_data/training_isds_cropped"

# Create dataset
dataset = ImageDatasetGenerator(image_folder, 
                                isd_map_folder, 
                                split = None, 
                                val_size = 0.2, 
                                random_seed = 42, 
                                transform_images = transform_images, 
                                transform_guidance = transform_guidance)

# Create dataset
train_dataset = ImageDatasetGenerator(image_folder, 
                                      isd_map_folder, 
                                      split = "train", 
                                      val_size = 0.2, 
                                      random_seed = 42, 
                                      transform_images = transform_images, 
                                      transform_guidance = transform_guidance)

# Create dataset
val_dataset = ImageDatasetGenerator(image_folder, 
                                    isd_map_folder, 
                                    split = "val", 
                                    val_size = 0.2, 
                                    random_seed = 42, 
                                    transform_images = transform_images, 
                                    transform_guidance = transform_guidance)

# Create dataloader
# Shuffle equals false for now for testing
# drop last creates only full batches
dataloader = DataLoader(val_dataset, batch_size = 4, shuffle = False, drop_last = True)

Assertion Passed!!! We have the same number of images and ISD maps.
Assertion Passed!!! We have the same number of images and ISD maps.
Assertion Passed!!! We have the same number of images and ISD maps.


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'  
embed_size = 64
num_class = 10
model = CvT(embed_size).to(device)

Entering embedding: Patch Size: 7 -- Stride: 4 -- Embedding Dims 64
Entering embedding: Patch Size: 3 -- Stride: 2 -- Embedding Dims 192
Entering embedding: Patch Size: 3 -- Stride: 2 -- Embedding Dims 384


In [8]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Parameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 10
learning_rate = 0.001
batch_size = 4

# Define loss function and optimizer
criterion = nn.MSELoss()  # Assuming regression; adjust for classification if needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()
    train_loss = 0.0

    # Training Step
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    for batch in tqdm(train_loader, desc="Training"):
        images, guidance_maps = batch
        images, guidance_maps = images.to(device), guidance_maps.to(device)

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, guidance_maps)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation Step
    model.eval()
    val_loss = 0.0
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            images, guidance_maps = batch
            images, guidance_maps = images.to(device), guidance_maps.to(device)

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = criterion(outputs, guidance_maps)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss:.4f}")

print("Training Complete!")


Epoch 1/10


Training:   0%|          | 0/298 [00:00<?, ?it/s]

Shape X IN: torch.Size([4, 3, 224, 224])
Shape X OUT: torch.Size([4, 3025, 64])
Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Shape X IN: torch.Size([4, 64, 55, 55])
Shape X OUT: torch.Size([4, 729, 192])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Shape X IN: torch.Size([4, 192, 27, 27])
Shape X OUT: torch.Size([4, 169, 384])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
After Stage 3 -> 2: torch.Size([4, 192, 27, 27])
After Stage 2 -> 1: torch.Size([4, 64, 55, 55])
After Stage 1 -> Original: torch.Size([4, 3, 224, 224])


Training:   0%|          | 1/298 [00:01<06:13,  1.26s/it]

Shape X IN: torch.Size([4, 3, 224, 224])
Shape X OUT: torch.Size([4, 3025, 64])
Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Shape X IN: torch.Size([4, 64, 55, 55])
Shape X OUT: torch.Size([4, 729, 192])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Shape X IN: torch.Size([4, 192, 27, 27])
Shape X OUT: torch.Size([4, 169, 384])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
After Stage 3 -> 2: torch.Size([4, 192, 27, 27])
After Stage 2 -> 1: torch.Size([4, 64, 55, 55])
After Stage 1 -> Original: torch.Size([4, 3, 224, 224])


...

Shape X IN: torch.Size([4, 3, 224, 224])
Shape X OUT: torch.Size([4, 3025, 64])
Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Shape X IN: torch.Size([4, 64, 55, 55])
Shape X OUT: torch.Size([4, 729, 192])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Shape X IN: torch.Size([4, 192, 27, 27])
Shape X OUT: torch.Size([4, 169, 384])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
After Stage 3 -> 2: torch.Size([4, 192, 27, 27])
After Stage 2 -> 1: torch.Size([4, 64, 55, 55])
After Stage 1 -> Original: torch.Size([4, 3, 224, 224])


Training:  12%|█▏        | 36/298 [00:36<04:24,  1.01s/it]

Shape X IN: torch.Size([4, 3, 224, 224])
Shape X OUT: torch.Size([4, 3025, 64])
Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Shape X IN: torch.Size([4, 64, 55, 55])
Shape X OUT: torch.Size([4, 729, 192])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Shape X IN: torch.Size([4, 192, 27, 27])
Shape X OUT: torch.Size([4, 169, 384])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
After Stage 3 -> 2: torch.Size([4, 192, 27, 27])
After Stage 2 -> 1: torch.Size([4, 64, 55, 55])
After Stage 1 -> Original: torch.Size([4, 3, 224, 224])


Training:  12%|█▏        | 37/298 [00:37<04:23,  1.01s/it]

Shape X IN: torch.Size([4, 3, 224, 224])
Shape X OUT: torch.Size([4, 3025, 64])
Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Shape X IN: torch.Size([4, 64, 55, 55])
Shape X OUT: torch.Size([4, 729, 192])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Shape X IN: torch.Size([4, 192, 27, 27])
Shape X OUT: torch.Size([4, 169, 384])
